In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_selection import mutual_info_regression, mutual_info_classif
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV, PredefinedSplit, GridSearchCV
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [ ]:
##############################################################################
# 1) Define global resources so we can reuse the same objects for train & test
##############################################################################

# --- Ordinal mapping for your ordinal columns ---
ordinal_mapping = {
    "GarageQual":    {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
    "Fence":         {"GdPrv": 4, "MnPrv": 3, "GdWo": 2, "MnWw": 1, "NA": 0},
    "GarageFinish":  {"Fin": 3, "RFn": 2, "Unf": 1, "NA": 0},
    "KitchenQual":   {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0},
    "GarageCond":    {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
    "HeatingQC":     {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0},
    "ExterQual":     {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0},
    "BsmtCond":      {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
    "LandSlope":     {"Gtl": 2, "Mod": 1, "Sev": 0},
    "ExterCond":     {"Ex": 4, "Gd": 3, "TA": 2, "Fa": 1, "Po": 0},
    "BsmtExposure":  {"Gd": 4, "Av": 3, "Mn": 2, "No": 1, "NA": 0},
    "PavedDrive":    {"Y": 2, "P": 1, "N": 0},
    "BsmtQual":      {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
    "LotShape":      {"Reg": 3, "IR1": 2, "IR2": 1, "IR3": 0},
    "BsmtFinType2":  {"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1, "NA": 0},
    "BsmtFinType1":  {"GLQ": 6, "ALQ": 5, "BLQ": 4, "Rec": 3, "LwQ": 2, "Unf": 1, "NA": 0},
    "FireplaceQu":   {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
    "Utilities":     {"AllPub": 3, "NoSewr": 2, "NoSeWa": 1, "ELO": 0},
    "Functional":    {"Typ": 7, "Min1": 6, "Min2": 5, "Mod": 4, "Maj1": 3,
                      "Maj2": 2, "Sev": 1, "Sal": 0},
    "PoolQC":        {"Ex": 5, "Gd": 4, "TA": 3, "Fa": 2, "Po": 1, "NA": 0},
}
# Extract the list of ordinal columns from that mapping
ordinal_cols = list(ordinal_mapping.keys())

# --- Nominal features (for One-Hot Encoding) ---
nominal_features = [
    "MSZoning", "Street", "Alley", "LandContour", "LotConfig", "Neighborhood",
    "Condition1", "Condition2", "BldgType", "HouseStyle", "RoofStyle", "RoofMatl",
    "Exterior1st", "Exterior2nd", "MasVnrType", "Foundation", "Heating",
    "CentralAir", "Electrical", "GarageType", "MiscFeature", "SaleType",
    "SaleCondition"
]

# Create ONE global OrdinalEncoder and ONE global OneHotEncoder.
# We'll fit them on train, then reuse them to transform test.
ordinal_encoder = OrdinalEncoder(
    categories=[list(ordinal_mapping[col].keys()) for col in ordinal_cols],
    handle_unknown="use_encoded_value",
    unknown_value=-1
)
one_hot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

##############################################################################
# 2) Main preprocessing function
##############################################################################
def preprocess_data(df, is_train=True):
    """
    Preprocess a housing DataFrame.
    If is_train=True, drops outliers and fits the encoders.
    If is_train=False, only transforms using the fitted encoders (NO outlier removal).

    Returns:
      (X, y) if is_train=True
      X if is_train=False
    """

    # Make a copy so we don't overwrite the original data
    df = df.copy()

    # If this is training data, grab the target 'SalePrice'
    # and remove it from the DataFrame.
    if is_train:
        y = df['SalePrice']
        df.drop('SalePrice', axis=1, inplace=True)
    else:
        y = None  # no target in test set

    # 1) Convert certain columns to numeric if they exist
    cols_to_numeric = [
        "LotFrontage", "MasVnrArea", "BsmtFinSF1", "BsmtFinSF2", "BsmtUnfSF",
        "TotalBsmtSF", "BsmtFullBath", "GarageCars", "BsmtHalfBath", "GarageYrBlt"
    ]
    for col in cols_to_numeric:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")

    # 2) Fill numeric columns with 0, categorical with "NA"
    numeric_cols = df.select_dtypes(include=['number']).columns
    df[numeric_cols] = df[numeric_cols].fillna(0)

    # Identify any columns of object/string dtype
    cat_cols = df.select_dtypes(include=['object']).columns
    for cat_col in cat_cols:
        df[cat_col] = df[cat_col].fillna("NA")

    # 3) Feature engineering
    # Existing Feature Engineering
    if set(["FullBath","HalfBath","BsmtFullBath","BsmtHalfBath"]).issubset(df.columns):
        df["TotalBath"] = (
            df["FullBath"] +
            0.5 * df["HalfBath"] +
            df["BsmtFullBath"] +
            0.5 * df["BsmtHalfBath"]
        )
    else:
        df["TotalBath"] = 0

    if "GarageYrBlt" in df.columns:
        df["Has_garage"] = df["GarageYrBlt"].notnull().astype(int)

    if set(["YrSold","YearBuilt"]).issubset(df.columns):
        df["House_Age"] = df["YrSold"] - df["YearBuilt"]

    if set(["YearBuilt","YearRemodAdd"]).issubset(df.columns):
        df["Is_Remodeled"] = (df["YearBuilt"] != df["YearRemodAdd"]).astype(int)

    # New Feature Engineering
    if set(["Street", "Alley", "MasVnrType", "GarageType", "MiscFeature",
            "BsmtQual", "FireplaceQu", "PoolQC", "Fence", "CentralAir"]).issubset(df.columns):
        # Count missing values in the selected columns and add penalties if certain conditions are met
        df["Lack_of_feature_index"] = df[["Street", "Alley", "MasVnrType", "GarageType", "MiscFeature",
                                           "BsmtQual", "FireplaceQu", "PoolQC", "Fence"]].isnull().sum(axis=1) + \
                                       (df["MasVnrType"] == 'None') + (df["CentralAir"] == 'No')
    else:
        df["Lack_of_feature_index"] = 0

    for col in ["PoolQC", "MiscFeature", "Fence"]:
        if col not in df.columns:
            df[col] = pd.NA

    df["MiscFeatureExtended"] = (
        df["PoolQC"].notnull().astype(int) +
        df["MiscFeature"].notnull().astype(int) +
        df["Fence"].notnull().astype(int)
    ).astype(int)

    if "Alley" in df.columns:
        df["Has_Alley"] = df["Alley"].notnull().astype(int)
    else:
        df["Has_Alley"] = 0

    if set(["GrLivArea", "LotArea"]).issubset(df.columns):
        df["Lot_occupation"] = df["GrLivArea"] / df["LotArea"]
    else:
        df["Lot_occupation"] = 0

    if set(["TotalBsmtSF", "1stFlrSF", "2ndFlrSF"]).issubset(df.columns):
        df["Number_of_floors"] = (
            (df["TotalBsmtSF"] != 0).astype(int) +
            (df["1stFlrSF"] != 0).astype(int) +
            (df["2ndFlrSF"] != 0).astype(int)
        )
    else:
        df["Number_of_floors"] = 0

    if set(["GrLivArea", "TotalBsmtSF"]).issubset(df.columns):
        df['Total_Close_Live_Area'] = df['GrLivArea'] + df['TotalBsmtSF']
    else:
        df['Total_Close_Live_Area'] = 0

    if set(["WoodDeckSF", "OpenPorchSF", "EnclosedPorch", "3SsnPorch", "ScreenPorch"]).issubset(df.columns):
        df['Outside_live_area'] = (
            df['WoodDeckSF'] +
            df['OpenPorchSF'] +
            df['EnclosedPorch'] +
            df['3SsnPorch'] +
            df['ScreenPorch']
        )
    else:
        df['Outside_live_area'] = 0

    df['Total_usable_area'] = df.get('Total_Close_Live_Area', 0) + df.get('Outside_live_area', 0)

    if "OverallQual" in df.columns and "Total_usable_area" in df.columns:
        df['Area_Quality_Indicator'] = df['Total_usable_area'] * df['OverallQual']
    else:
        df['Area_Quality_Indicator'] = 0

    if set(["OverallQual", "OverallCond", "Total_usable_area"]).issubset(df.columns):
        df['Area_Qual_Cond_Indicator'] = df['Total_usable_area'] * df['OverallQual'] * df['OverallCond']
    else:
        df['Area_Qual_Cond_Indicator'] = 0

    if "BsmtQual" in df.columns:
        df['HasBsmt'] = df['BsmtQual'].notnull().astype(int)
    else:
        df['HasBsmt'] = 0

    if set(["OverallQual", "OverallCond"]).issubset(df.columns):
        df['Quality_conditition'] = df['OverallQual'] * df['OverallCond']
        df['Quality_conditition_2'] = df['OverallQual'] + df['OverallCond']
    else:
        df['Quality_conditition'] = 0
        df['Quality_conditition_2'] = 0

    if set(["YrSold", "YearRemodAdd"]).issubset(df.columns):
        df['House_Age2'] = df['YrSold'] - df['YearRemodAdd']
    else:
        df['House_Age2'] = 0

    # 4) Outlier removal: only do this on TRAIN.
    if is_train:
        # (a) LotFrontage > 200
        if "LotFrontage" in df.columns:
            idx = df[df["LotFrontage"] > 200].index
            df.drop(idx, inplace=True)
            y.drop(idx, inplace=True)

        # (b) LotArea > 100000
        if "LotArea" in df.columns:
            idx = df[df["LotArea"] > 100000].index
            df.drop(idx, inplace=True)
            y.drop(idx, inplace=True)

        # (c) BsmtFinSF1 > 4000
        if "BsmtFinSF1" in df.columns:
            idx = df[df["BsmtFinSF1"] > 4000].index
            df.drop(idx, inplace=True)
            y.drop(idx, inplace=True)

        # (d) TotalBsmtSF > 6000
        if "TotalBsmtSF" in df.columns:
            idx = df[df["TotalBsmtSF"] > 6000].index
            df.drop(idx, inplace=True)
            y.drop(idx, inplace=True)

        # (e) 1stFlrSF > 4000
        if "1stFlrSF" in df.columns:
            idx = df[df["1stFlrSF"] > 4000].index
            df.drop(idx, inplace=True)
            y.drop(idx, inplace=True)

        # (f) GrLivArea > 4000 & y < 300000
        if "GrLivArea" in df.columns:
            idx = df[(df["GrLivArea"] > 4000) & (y < 300000)].index
            df.drop(idx, inplace=True)
            y.drop(idx, inplace=True)

        # (g) LowQualFinSF > 550
        if "LowQualFinSF" in df.columns:
            idx = df[df["LowQualFinSF"] > 550].index
            df.drop(idx, inplace=True)
            y.drop(idx, inplace=True)

    # 5) Ordinal Encoding for the columns in ordinal_mapping
    existing_ordinal = [col for col in ordinal_cols if col in df.columns]

    # Clean up whitespace if any
    for col in existing_ordinal:
        df[col] = df[col].astype(str).str.strip()

    if len(existing_ordinal) > 0:
        if is_train:
            df[existing_ordinal] = ordinal_encoder.fit_transform(df[existing_ordinal])
        else:
            df[existing_ordinal] = ordinal_encoder.transform(df[existing_ordinal])
        # Replace unknown encodings (-1) with 0
        df.replace(-1, 0, inplace=True)

    # 6) One-Hot Encoding for nominal features
    existing_nominal = [col for col in nominal_features if col in df.columns]

    if len(existing_nominal) > 0:
        if is_train:
            # Fit on train
            encoded = one_hot_encoder.fit_transform(df[existing_nominal])
        else:
            # Transform test
            encoded = one_hot_encoder.transform(df[existing_nominal])

        # Convert to DataFrame
        encoded_df = pd.DataFrame(
            encoded,
            columns=one_hot_encoder.get_feature_names_out(existing_nominal),
            index=df.index
        )
        # Drop original nominal cols, then join
        df.drop(columns=existing_nominal, inplace=True)
        df = df.join(encoded_df)

    # Return final results
    if is_train:
        return df, y
    else:
        return df


In [3]:
df_train = pd.read_csv("train.csv")
X_train, y_train = preprocess_data(df_train, is_train=True)

In [4]:
df_test = pd.read_csv("test.csv")
test_ids = df_test['Id'].copy()
X_test = preprocess_data(df_test, is_train=False)

In [5]:
print(f"New train shape: {X_train.shape}, test shape: {X_test.shape}")

New train shape: (1452, 244), test shape: (1459, 244)


## Random Forest

In [11]:
param_dist = {
    'n_estimators': randint(100, 2000),        # Number of trees
    'max_depth': randint(1, 20),             # Maximum tree depth
    'min_samples_split': randint(2, 20),     # Minimum samples required to split an internal node
    'min_samples_leaf': randint(1, 10),      # Minimum samples required at a leaf node
    'max_features': ['sqrt', 'log2', None], # Number of features to consider at each split
    'criterion': ['squared_error'] # For regression
}


In [12]:
# Initialize the RandomForestRegressor
rf = RandomForestRegressor(bootstrap=True,oob_score=True, random_state=42)


In [13]:
# cv_dummy = [(np.arange(len(X_train)), np.arange(len(X_train)))]
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=15,
    scoring='neg_root_mean_squared_error',
    random_state=42,
    n_jobs=1,
    verbose=5,
    return_train_score=True,
    cv=3
)

In [14]:
random_search.fit(X_train, y_train)

Fitting 3 folds for each of 15 candidates, totalling 45 fits
[CV 1/3] END criterion=squared_error, max_depth=7, max_features=sqrt, min_samples_leaf=8, min_samples_split=8, n_estimators=221;, score=(train=-25712.192, test=-23942.048) total time=   0.4s
[CV 2/3] END criterion=squared_error, max_depth=7, max_features=sqrt, min_samples_leaf=8, min_samples_split=8, n_estimators=221;, score=(train=-24199.166, test=-32127.983) total time=   0.4s
[CV 3/3] END criterion=squared_error, max_depth=7, max_features=sqrt, min_samples_leaf=8, min_samples_split=8, n_estimators=221;, score=(train=-24786.899, test=-29945.541) total time=   0.3s
[CV 1/3] END criterion=squared_error, max_depth=19, max_features=None, min_samples_leaf=8, min_samples_split=5, n_estimators=971;, score=(train=-22654.649, test=-26139.800) total time=  13.0s
[CV 2/3] END criterion=squared_error, max_depth=19, max_features=None, min_samples_leaf=8, min_samples_split=5, n_estimators=971;, score=(train=-21113.750, test=-28738.220) t

RandomizedSearchCV(cv=3,
                   estimator=RandomForestRegressor(oob_score=True,
                                                   random_state=42),
                   n_iter=15, n_jobs=1,
                   param_distributions={'criterion': ['squared_error'],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000181B4D01C90>,
                                        'max_features': ['sqrt', 'log2', None],
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000181B50FAC10>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000181B50FB290>,
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000181A1A05890>},
                   random_state=42, return_train_score=True,
                   scoring='neg_root_mean_squared_error', verbose=5)

In [15]:
# Get the best parameters and model
best_params = random_search.best_params_
best_model = random_search.best_estimator_
best_r2 = -random_search.best_score_
print("Random Forest best params:", best_params)
print("Random Forest best R² on validation set: {:.4f}".format(best_r2))

Random Forest best params: {'criterion': 'squared_error', 'max_depth': 9, 'max_features': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 700}
Random Forest best R² on validation set: 25867.0236


In [16]:
# Train a final RandomForestRegressor using the best parameters
final_model = RandomForestRegressor(**best_params,bootstrap=True,oob_score=True, random_state=42)
final_model.fit(X_train, y_train)

RandomForestRegressor(max_depth=9, max_features=None, min_samples_leaf=2,
                      min_samples_split=5, n_estimators=700, oob_score=True,
                      random_state=42)

In [17]:
# Generate predictions for the test set
test_predictions = final_model.predict(X_test)

In [18]:
# Create a submission dataframe (assuming the sample_submission.csv expects 'Id' and 'SalePrice')
submission = pd.DataFrame({'Id': test_ids, 'SalePrice': test_predictions})
submission.to_csv('submission_Random_forest.csv', index=False)

print("Submission file 'submission.csv' created successfully!")

Submission file 'submission.csv' created successfully!


## XGBOOST

In [19]:
param_grid = {
    'n_estimators': list(range(1000, 1601, 50)),
    'max_depth': randint(2, 7),
    'learning_rate': [0.01, 0.03, 0.1, 0.3, 0.05, 0.5],
    'subsample': [0.7, 0.6, 0.5, 0.4, 0.3],
    'colsample_bytree': [0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1],
}
# param_grid = {
#     'n_estimators': randint(50, 1000)
# }

In [20]:
xgb_reg = xgb.XGBRegressor(
    objective='reg:squarederror',
    random_state=42,
)

In [21]:
# Set up RandomizedSearchCV using negative RMSE as the scoring metric
random_search_xgb = RandomizedSearchCV(estimator=xgb_reg,
                                   param_distributions=param_grid,
                                   n_iter=50,
                                   cv=4,
                                   scoring='neg_root_mean_squared_error',
                                   random_state=42,
                                   n_jobs=1,
                                   verbose=2)

In [22]:
# Run the search
random_search_xgb.fit(X_train, y_train)

Fitting 4 folds for each of 50 candidates, totalling 200 fits
[CV] END colsample_bytree=0.1, learning_rate=0.3, max_depth=6, n_estimators=1500, subsample=0.3; total time=   4.2s
[CV] END colsample_bytree=0.1, learning_rate=0.3, max_depth=6, n_estimators=1500, subsample=0.3; total time=   3.8s
[CV] END colsample_bytree=0.1, learning_rate=0.3, max_depth=6, n_estimators=1500, subsample=0.3; total time=   3.6s
[CV] END colsample_bytree=0.1, learning_rate=0.3, max_depth=6, n_estimators=1500, subsample=0.3; total time=   3.2s
[CV] END colsample_bytree=0.3, learning_rate=0.03, max_depth=4, n_estimators=1300, subsample=0.5; total time=   2.4s
[CV] END colsample_bytree=0.3, learning_rate=0.03, max_depth=4, n_estimators=1300, subsample=0.5; total time=   2.4s
[CV] END colsample_bytree=0.3, learning_rate=0.03, max_depth=4, n_estimators=1300, subsample=0.5; total time=   2.5s
[CV] END colsample_bytree=0.3, learning_rate=0.03, max_depth=4, n_estimators=1300, subsample=0.5; total time=   2.8s
[CV] E

RandomizedSearchCV(cv=4,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=...
                   param_distributions={'colsample_bytree': [0.7, 0.6, 0.5, 0.4,
                                                             0.3, 0.2, 0.1],
                                        'learning_rate': [0.01, 0.03, 0.1, 0.3,
                                                          0.05, 0.5],
                                        'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x00000181B5177710>,
                                        'n_estimators': [1000, 1050, 1100, 1150,
                                                         1200, 1250, 1300, 1350,
                                                         1400, 1450, 1500, 1550,
                                                         1600],
                                        'subsample': [0.7, 0.6, 0.5, 0.4, 0.3]},
                   random_state=42, scoring='neg_root_mean_squared_error',
                   verbose=2)

In [23]:
# Get the best parameters and model
best_params_xgb = random_search_xgb.best_params_
best_model_xgb = random_search_xgb.best_estimator_
best_rmse_xgb = -random_search_xgb.best_score_
print("XGBoost best params:", best_params_xgb)
print("XGBoost best RMSE on validation set: {:.4f}".format(best_rmse_xgb))

XGBoost best params: {'colsample_bytree': 0.2, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 1350, 'subsample': 0.4}
XGBoost best RMSE on validation set: 21087.7861


In [24]:
# Evaluate on the validation set
# val_predictions = best_model.predict(X_val)
# rmse_val = np.sqrt(mean_squared_error(y_val, val_predictions))
# print("Best Parameters:", best_params)
# print("Validation RMSE:", rmse_val)

In [25]:
# Train the final XGBRegressor using the best parameters
final_model_xgb = xgb.XGBRegressor(**best_params_xgb, objective='reg:squarederror', random_state=42)
final_model_xgb.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.2, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=3, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=1350, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [26]:
# Generate predictions for the test set
test_predictions_xgb = final_model_xgb.predict(X_test)

In [27]:
# Create a submission dataframe (assumes sample_submission.csv expects 'Id' and 'SalePrice')
submission = pd.DataFrame({'Id': test_ids, 'SalePrice': test_predictions_xgb})
submission.to_csv('submission_XGBoost.csv', index=False)

print("Submission file 'submission_XGBoost.csv' created successfully!")

Submission file 'submission_XGBoost.csv' created successfully!


## Linear Regression

In [28]:
param_dist_ridge = {
    'alpha': uniform(0.01, 100)   # Sample alpha between 0.01 and 100
}

In [ ]:
ridge = Ridge()

random_search_ridge = RandomizedSearchCV(
    estimator=ridge,
    param_distributions=param_dist_ridge,
    n_iter=50,
    cv=5,
    scoring='neg_root_mean_squared_error',
    random_state=42,
    n_jobs=1,
    verbose=2,
    return_train_score=True
)


In [30]:
random_search_ridge.fit(X_train, y_train)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV] END ...........................alpha=37.464011884736244; total time=   0.0s
[CV] END ...........................alpha=37.464011884736244; total time=   0.0s
[CV] END ...........................alpha=37.464011884736244; total time=   0.0s
[CV] END ...........................alpha=37.464011884736244; total time=   0.0s
[CV] END ...........................alpha=37.464011884736244; total time=   0.0s
[CV] END ............................alpha=95.08143064099163; total time=   0.0s
[CV] END ............................alpha=95.08143064099163; total time=   0.0s
[CV] END ............................alpha=95.08143064099163; total time=   0.0s
[CV] END ............................alpha=95.08143064099163; total time=   0.0s
[CV] END ............................alpha=95.08143064099163; total time=   0.0s
[CV] END ............................alpha=73.20939418114051; total time=   0.0s
[CV] END ............................alpha=73.2

RandomizedSearchCV(cv=5, estimator=Ridge(), n_iter=50, n_jobs=1,
                   param_distributions={'alpha': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x00000181B70D7550>},
                   random_state=42, return_train_score=True,
                   scoring='neg_root_mean_squared_error', verbose=2)

In [31]:
# Best hyperparameters and score (convert negative RMSE to RMSE)
best_params_ridge = random_search_ridge.best_params_
best_rmse_ridge = -random_search_ridge.best_score_
print("Ridge Regression best params:", best_params_ridge)
print("Ridge Regression best RMSE on validation set: {:.4f}".format(best_rmse_ridge))

Ridge Regression best params: {'alpha': 21.243911067827618}
Ridge Regression best RMSE on validation set: 22271.8941


In [32]:
# Train final model on the original training set using best hyperparameters
final_ridge = Ridge(**best_params_ridge)
final_ridge.fit(X_train, y_train)

Ridge(alpha=21.243911067827618)

In [33]:
# Generate predictions for the test set
test_predictions_ridge = final_ridge.predict(X_test)

In [34]:
# Create a submission dataframe (assumes sample_submission.csv expects 'Id' and 'SalePrice')
submission = pd.DataFrame({'Id': test_ids, 'SalePrice': test_predictions_ridge})
submission.to_csv('submission_ridge.csv', index=False)

print("Submission file 'submission_ridge.csv' created successfully!")

Submission file 'submission_ridge.csv' created successfully!


## SVM

In [35]:
#param_dist_svr = {
#    'C': uniform(0.1, 10),
#    'epsilon': uniform(0.01, 1),
#    'kernel': ['rbf', 'linear']
#}

In [36]:
param_grid_svr = {
    'C': [0.1, 1, 3, 10],
    'epsilon': [0.01, 0.1, 0.3, 1],
    'kernel': ['rbf', 'linear']
}

In [37]:
svr = SVR()

In [38]:
# random_search_svr = RandomizedSearchCV(
#    estimator=svr,
#    param_distributions=param_dist_svr,
#    n_iter=50,
#    cv=5,
#    scoring='neg_root_mean_squared_error',
#    random_state=42,
#    n_jobs=-1,
#    verbose=2,
#    return_train_score=True
#)

In [39]:
# Set up GridSearchCV
grid_search_svr = GridSearchCV(
    estimator=svr,
    param_grid=param_grid_svr,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=2
)

In [ ]:
grid_search_svr.fit(X_train, y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] END ....................C=0.1, epsilon=0.01, kernel=rbf; total time=   0.1s
[CV] END ....................C=0.1, epsilon=0.01, kernel=rbf; total time=   0.1s
[CV] END ....................C=0.1, epsilon=0.01, kernel=rbf; total time=   0.1s
[CV] END .................C=0.1, epsilon=0.01, kernel=linear; total time=  45.9s
[CV] END .................C=0.1, epsilon=0.01, kernel=linear; total time=  51.3s
[CV] END .................C=0.1, epsilon=0.01, kernel=linear; total time= 1.8min
[CV] END .....................C=0.1, epsilon=0.1, kernel=rbf; total time=   0.2s
[CV] END .....................C=0.1, epsilon=0.1, kernel=rbf; total time=   0.2s
[CV] END .....................C=0.1, epsilon=0.1, kernel=rbf; total time=   0.1s
[CV] END ..................C=0.1, epsilon=0.1, kernel=linear; total time=  59.8s
[CV] END ..................C=0.1, epsilon=0.1, kernel=linear; total time= 1.3min
[CV] END ..................C=0.1, epsilon=0.1, k

In [ ]:
# Fit the randomized search on the combined dataset
# random_search_svr.fit(X_train, y_train)

In [ ]:
# best_params_svr = random_search_svr.best_params_
# best_rmse_svr = -random_search_svr.best_score_
# print("SVR best params:", best_params_svr)
# print("SVR best RMSE on validation set: {:.4f}".format(best_rmse_svr))

In [ ]:
best_params_svr = grid_search_svr.best_params_
best_rmse_svr = -grid_search_svr.best_score_
print("Best Parameters:", best_params_svr)
print("Best RMSE:", best_rmse_svr)

In [ ]:
final_svr = SVR(**best_params_svr)
final_svr.fit(X_train, y_train)

In [ ]:
# Generate predictions for the test set
test_predictions_svr = final_svr.predict(X_test)

In [ ]:

submission = pd.DataFrame({'Id': test_ids, 'SalePrice': test_predictions_svr})
submission.to_csv('submission_svr.csv', index=False)

print("Submission file 'submission_svr.csv' created successfully!")

## Final Results

In [ ]:
best_models = {
    'Random Forest': random_search.best_estimator_,
    'XGBoost': random_search_xgb.best_estimator_,
    'Linear Regression': random_search_ridge.best_estimator_,
    'SVM': grid_search_svr.best_estimator_
}

In [ ]:
# Prepare a list to store results
results = []

# Evaluate each model on the validation set
for model_name, model in best_models.items():
    y_pred = model.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    # Convert the best params dictionary to a formatted string for display
    best_params = model.get_params()
    best_params_str = ", ".join([f"{k}={v}" for k, v in best_params.items() if k in best_models[model_name].get_params()])

    results.append({
        'Model': model_name,
        'RMSE': rmse,
        'Best Parameters': best_params_str
    })

# Create a DataFrame to display the results in a table format
results_df = pd.DataFrame(results)
print(results_df)

## Final Results

In [ ]:
models = {
    "Random Forest": (random_search.best_estimator_, random_search.best_params_),
    "XGBoost": (random_search_xgb.best_estimator_, random_search_xgb.best_params_),
    "Linear Regression": (random_search_ridge.best_estimator_, random_search_ridge.best_params_),
    "SVM": (grid_search_svr.best_estimator_, grid_search_svr.best_params_)
}

# Prepare a list to store the evaluation results
results = []

# Evaluate each model on the validation set
for model_name, (model, best_params) in models.items():
    # Predict on the validation set
    y_pred = model.predict(X_val)
    # Compute RMSE (ensure it's positive)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    # Append the model results, including the best parameters dictionary
    results.append({
        "Model": model_name,
        "RMSE": rmse,
        "Best Parameters": best_params
    })

# Create a DataFrame to display the results in a neat table
results_df = pd.DataFrame(results)
print(results_df)